In [1]:
# Clone AnimateDiff repo
!git clone https://github.com/Kyotogod/AnimateDiff-Colab.git
%cd AnimateDiff-Colab

# Install dependencies
!pip install -r requirements.txt

# Download AnimateDiff motion module (already included in notebook below, optional here)
!mkdir -p models/Motion_Module
!wget -P models/Motion_Module https://huggingface.co/guoyww/AnimateDiff/resolve/main/mm_sd_v15_v2.ckpt

# Download Stable Diffusion 1.5 model (anime one recommended)
!mkdir -p models/StableDiffusion
!wget -P models/StableDiffusion https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/model.ckpt


Cloning into 'AnimateDiff-Colab'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'AnimateDiff-Colab'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
--2025-03-30 03:09:45--  https://huggingface.co/guoyww/AnimateDiff/resolve/main/mm_sd_v15_v2.ckpt
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.23, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt [following]
--2025-03-30 03:09:45--  https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/80/a9/80a9efd8e3732d097f2777edd953760ce288e2370fb8a8bb4803c0f8f29dca40/69e

In [2]:
# Install ffmpeg if not already installed
!apt-get -y install ffmpeg

# Create the transition video
!ffmpeg -framerate 12 -i frames/frame_%03d.png -c:v libx264 -pix_fmt yuv420p transition.mp4


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [7]:
from PIL import Image
import os
start_img_path = "/content/panel.jpg"
end_img_path = "/content/end.png"
start_img = Image.open(start_img_path).resize((512, 512))
end_img = Image.open(end_img_path).resize((512, 512))

def interpolate_images(img1, img2, steps):
    img_seq = []
    for i in range(steps):
        alpha = i / (steps - 1)
        blended = Image.blend(img1, img2, alpha)
        img_seq.append(blended)
    return img_seq

frames = interpolate_images(start_img, end_img, steps=16)

# Save frames
os.makedirs("frames", exist_ok=True)
for i, frame in enumerate(frames):
    frame.save(f"frames/frame_{i:03d}.png")

In [8]:
!ffmpeg -framerate 12 -i frames/frame_%03d.png -c:v libx264 -pix_fmt yuv420p transition.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [9]:
!pip install diffusers transformers accelerate ftfy
!pip install git+https://github.com/openai/CLIP.git
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [10]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import numpy as np
import os

# Load SD1.5 from HuggingFace
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

pipe.enable_attention_slicing()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
def image_to_latent(image_path):
    image = Image.open(image_path).convert("RGB").resize((512, 512))
    image = pipe.feature_extractor(images=image, return_tensors="pt").pixel_values.to("cuda", dtype=torch.float16)
    latents = pipe.vae.encode(image).latent_dist.sample() * 0.18215
    return latents

latent_start = image_to_latent("/content/panel.jpg")
latent_end = image_to_latent("/content/end.png")

def interpolate_latents(start, end, steps=16):
    return [(1 - t) * start + t * end for t in np.linspace(0, 1, steps)]

latent_seq = interpolate_latents(latent_start, latent_end, steps=16)



In [13]:
def latent_to_image(latent):
    latent = latent / 0.18215
    image = pipe.vae.decode(latent).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    # image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()[0]

    return (image * 255).astype(np.uint8)

os.makedirs("transition_frames", exist_ok=True)

for i, latent in enumerate(latent_seq):
    img = latent_to_image(latent)
    Image.fromarray(img).save(f"transition_frames/frame_{i:03d}.png")


In [14]:
!ffmpeg -framerate 8 -i transition_frames/frame_%03d.png -c:v libx264 -pix_fmt yuv420p transition2.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab